# Part 1 solutions (don't peek!)

_(Solutions for [Part 1 project](project.ipynb).)_

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

<br><br><br>

## Definitions

These two cells just bring back the `new_world` and `show` functions.

In [ ]:
WIDTH = 128
HEIGHT = 32

def new_world():
    world = np.zeros((HEIGHT, WIDTH), dtype=np.int32)

    for x, y in [
        ( 4, 125), ( 3, 124), ( 3, 123), ( 3, 122), ( 3, 121), ( 3, 120), ( 3, 119), ( 4, 119), ( 5, 119), ( 6, 120),
        (10, 121), (11, 120), (12, 119), (12, 120), (13, 120), (13, 121), (14, 121),
        (20, 121), (19, 120), (18, 120), (18, 119), (17, 121), (17, 120), (16, 121),
        (26, 125), (27, 124), (27, 123), (27, 122), (27, 121), (27, 120), (27, 119), (26, 119), (25, 119), (24, 120)
    ]:
        world[x][y] = 1

    return world

world = new_world()

In [ ]:
def show(world):
    for row in world:
        print("|" + "".join("@" if cell else "." for cell in row) + "|")

show(world)

<br><br><br>

## SOLUTIONS TO THE EXERCISE

We can write a completely general solution based on the [np.roll](https://numpy.org/doc/stable/reference/generated/numpy.roll.html) function. This function replaces an array with one in which all elements are shifted to the left or right when `axis=1` and up and down when `axis=0`.

In [ ]:
eyeball = np.array([
    [    1,    1,    1,    1,    1,    1,    1],
    [    1,    1,    1,    1,    1,    1,    1],
    [    1,    1,    1, 9999,    1,    1,    1],
    [    1,    1, 9999, 9999, 9999,    1,    1],
    [    1,    1, 9999, 9999, 9999,    1,    1],
    [    1,    1, 9999, 9999, 9999,    1,    1],
    [    1,    1, 9999, 9999, 9999,    1,    1],
    [    1,    1, 9999, 9999,    1,    1,    1],
    [    1,    1, 9999, 9999,    1,    1,    1],
    [    1,    1,    1,    1,    1,    1,    1],
    [    1,    1,    1,    1,    1,    1,    1],
    [    1,    1,    1,    1,    1,    1,    1],
])

In [ ]:
# Look to the left...
np.roll(eyeball, -1, axis=1)

In [ ]:
# Look to the right...
np.roll(eyeball, 1, axis=1)

In [ ]:
# Look up...
np.roll(eyeball, -2, axis=0)

And it wraps around at the edges.

In [ ]:
# Ouch!
np.roll(eyeball, -4, axis=0)

This is useful because we need to calculate the number of live neighbors each cell has. NumPy's `+` operation adds cells of equal-shaped arrays that line up with one another; `np.roll` makes them line up.

For instance, how many neighbors do each of the 5 cells in this one-dimensional array have?

In [ ]:
one_dimensional = np.array([0, 0, 1, 0, 1, 0, 0])

| | index 0 | index 1 | index 2 | index 3 | index 4 | index 5 | index 6 |
|:-|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| **array values** | 0 | 0 | 1 | 0 | 1 | 0 | 0 |

| | index 0 | index 1 | index 2 | index 3 | index 4 | index 5 | index 6 |
|:-|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| **left neighbors** | 0 | 0 | 0 | 1 | 0 | 1 | 0 |
| **right neighbors** | 0 | 1 | 0 | 1 | 0 | 0 | 0 |
| **total neighbors** | 0 | 1 | 0 | 2 | 0 | 1 | 0 |

In [ ]:
left_neighbors = np.roll(one_dimensional, 1)
left_neighbors

In [ ]:
right_neighbors = np.roll(one_dimensional, -1)
right_neighbors

In [ ]:
left_or_right_neighbors = np.roll(one_dimensional, 1) + np.roll(one_dimensional, -1)
left_or_right_neighbors

<br><br><br>

Each two-dimensional cell has 8 neighbors, so we add up 8 rolls.

<center>
<img src="../img/Moore_neighborhood_with_cardinal_directions.svg" width="25%">
</center>

In [ ]:
def step_numpy(world):
    num_neighbors = np.zeros(world.shape, dtype=np.int32)                   # initialize neighbors count

    num_neighbors += np.roll(np.roll(world,  1, axis=0),  1, axis=1)        # add NW neighbor
    num_neighbors += np.roll(np.roll(world,  1, axis=0),  0, axis=1)        # add N neighbor
    num_neighbors += np.roll(np.roll(world,  1, axis=0), -1, axis=1)        # add NE neighbor

    num_neighbors += np.roll(np.roll(world,  0, axis=0),  1, axis=1)        # add W neighbor
    num_neighbors += np.roll(np.roll(world,  0, axis=0), -1, axis=1)        # add E neighbor

    num_neighbors += np.roll(np.roll(world, -1, axis=0),  1, axis=1)        # add SW neighbor
    num_neighbors += np.roll(np.roll(world, -1, axis=0),  0, axis=1)        # add S neighbor
    num_neighbors += np.roll(np.roll(world, -1, axis=0), -1, axis=1)        # add SE neighbor

    survivors = ((world == 1) & (num_neighbors > 1) & (num_neighbors < 4))  # old cells that survive
    births    = ((world == 0) & (num_neighbors == 3))                       # new cells that are born
    return (births | survivors)

With an array of the number of neighbors each cell has, we can apply the rules as logical operators.

<br>

Repeatedly evaluate the next Jupyter cell (control-enter) to animate.

In [ ]:
# world = new_world()       # uncomment to reset the world

world = step_numpy(world)
show(world)

<br><br><br>

And it's also more than 10× faster than the Python function.

In [ ]:
%%timeit

step_numpy(world)

<br><br><br>

## Did you just have to guess "np.roll"?

No!

There are other ways to solve this. `np.roll` handles the wrap-around boundaries well, but we don't really need that until iteration 237. If we're less concerned about the edges, we can do it with slices.

The problem remains one of calculating the number of live neighbors for each cell.

In [ ]:
one_dimensional = np.array([0, 0, 1, 0, 1, 0, 0])

| | index 0 | index 1 | index 2 | index 3 | index 4 | index 5 | index 6 |
|:-|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| **array values** | 0 | 0 | 1 | 0 | 1 | 0 | 0 |

| | index 0 | index 1 | index 2 | index 3 | index 4 | index 5 | index 6 |
|:-|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| **left neighbors** | ? | 0 | 0 | 1 | 0 | 1 | 0 |
| **right neighbors** | 0 | 1 | 0 | 1 | 0 | 0 | ? |
| **total neighbors** | ? | 1 | 0 | 2 | 0 | 1 | ? |

Note that we can get all but the first left neighbors by slicing off the last element, and we can get all but the last right neighbors by slicing off the first element.

The easiest way to add these is to allocate a zero array of the right size and add in each shifted slice.

In [ ]:
left_or_right_neighbors = np.zeros(7, np.int32)

left_or_right_neighbors[1:] += one_dimensional[:-1]
left_or_right_neighbors[:-1] += one_dimensional[1:]

left_or_right_neighbors

The two elements `left_or_right_neighbors[0]` and `left_or_right_neighbors[-1]` have not been assigned; they're still zero. We're assuming that the values that would wrap around are zero.

In the Game of Life, this makes the edges of the pitri dish deadly.

In [ ]:
def step_numpy_2(world):
    num_neighbors = np.zeros(world.shape, dtype=np.int32)                   # initialize neighbors count

    num_neighbors[1:  , 1:  ] += world[ :-1,  :-1]                          # add NW neighbor
    num_neighbors[1:  ,  :  ] += world[ :-1,  :  ]                          # add N neighbor
    num_neighbors[1:  ,  :-1] += world[ :-1, 1:  ]                          # add NE neighbor

    num_neighbors[ :  , 1:  ] += world[ :  ,  :-1]                          # add W neighbor
    num_neighbors[ :  ,  :-1] += world[ :  , 1:  ]                          # add E neighbor

    num_neighbors[ :-1, 1:  ] += world[1:  ,  :-1]                          # add SW neighbor
    num_neighbors[ :-1,  :  ] += world[1:  ,  :  ]                          # add S neighbor
    num_neighbors[ :-1,  :-1] += world[1:  , 1:  ]                          # add SE neighbor

    survivors = ((world == 1) & (num_neighbors > 1) & (num_neighbors < 4))  # old cells that survive
    births    = ((world == 0) & (num_neighbors == 3))                       # new cells that are born
    return (births | survivors)

<br><br><br>

Reset the world and check that `step_numpy_2` agrees with `step_numpy` for the first 200 steps.

In [ ]:
world = new_world()

for iteration in range(200):
    next_world = step_numpy_2(world)
    assert np.array_equal(next_world, step_numpy(world)), iteration
    world = next_world

Repeatedly evaluate the next Jupyter cell (control-enter) to animate.

In [ ]:
world = step_numpy_2(world)
show(world)

<br><br><br>

Although it's not a correct solution, if we care about edges, it happens to be much faster because we're not creating all the temporary `np.roll` outputs.

In [ ]:
%%timeit

step_numpy_2(world)

<br><br><br>

## Can we get the edges right, anyway?

Sure. It just involves careful bookkeeping.

Or making the `world` one cell larger along all edges, wrapping it explicitly, and then the fact that our slice-based calculation of `num_neighbors` excludes a one-cell wide edge would make it exactly right.

There's also a [np.pad](https://numpy.org/doc/stable/reference/generated/numpy.pad.html) function that would do the padding for us.

In [ ]:
def step_numpy_3(world):
    padded_world = np.empty((world.shape[0] + 2, world.shape[1] + 2), dtype=np.int32)

    padded_world[   0,    0] = world[-1, -1]
    padded_world[   0, 1:-1] = world[-1,  :]            # top of padded_world is bottom of world
    padded_world[   0,   -1] = world[-1,  0]

    padded_world[1:-1,    0] = world[ :, -1]            # left of padded_world is right of world
    padded_world[1:-1, 1:-1] = world                    # copy world into the center of padded_world
    padded_world[1:-1,   -1] = world[ :,  0]            # right of padded_world is left of world

    padded_world[  -1,    0] = world[ 0, -1]
    padded_world[  -1, 1:-1] = world[ 0,  :]            # bottom of padded_world is top of world
    padded_world[  -1,   -1] = world[ 0,  0]

    padded_num_neighbors = np.zeros(padded_world.shape, dtype=np.int32)     # initialize neighbors count

    padded_num_neighbors[1:  , 1:  ] += padded_world[ :-1,  :-1]            # add NW neighbor
    padded_num_neighbors[1:  ,  :  ] += padded_world[ :-1,  :  ]            # add N neighbor
    padded_num_neighbors[1:  ,  :-1] += padded_world[ :-1, 1:  ]            # add NE neighbor

    padded_num_neighbors[ :  , 1:  ] += padded_world[ :  ,  :-1]            # add W neighbor
    padded_num_neighbors[ :  ,  :-1] += padded_world[ :  , 1:  ]            # add E neighbor

    padded_num_neighbors[ :-1, 1:  ] += padded_world[1:  ,  :-1]            # add SW neighbor
    padded_num_neighbors[ :-1,  :  ] += padded_world[1:  ,  :  ]            # add S neighbor
    padded_num_neighbors[ :-1,  :-1] += padded_world[1:  , 1:  ]            # add SE neighbor

    num_neighbors = padded_num_neighbors[1:-1, 1:-1]

    survivors = ((world == 1) & (num_neighbors > 1) & (num_neighbors < 4))  # old cells that survive
    births    = ((world == 0) & (num_neighbors == 3))                       # new cells that are born
    return (births | survivors)

In [ ]:
world = new_world()

for iteration in range(1000):
    next_world = step_numpy_3(world)
    assert np.array_equal(next_world, step_numpy(world)), iteration
    world = next_world

<br><br><br>

And it's still faster than the `np.roll` solution because most of the assignments above are assignments of views, not newly allocated arrays.

In [ ]:
%%timeit

step_numpy_3(world)

<br><br><br>

## Did you try using SciPy's "convolve2d"?

In [ ]:
import scipy.signal

The exercise had a hint that this is solvable with [scipy.signal.convolve2d](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve2d.html). Here's how.

The number of neighbors is a sum over the 8 cells that border a given cell, not including that cell itself ("C" below).

<center>
<img src="../img/Moore_neighborhood_with_cardinal_directions.svg" width="25%">
</center>

This kind of sum, when applied to all cells in the grid, is a [convolution](https://en.wikipedia.org/wiki/Convolution).

The convolution kernel of the Game of Life is

In [ ]:
num_neighbors_convolver = np.array([[1, 1, 1],
                                    [1, 0, 1],
                                    [1, 1, 1]])

With this kernel, SciPy's `convolve2d` function counts the number of neighbors for us, for all cells in one function call.

In [ ]:
def step_scipy(world):
    num_neighbors = scipy.signal.convolve2d(world, num_neighbors_convolver, mode="same", boundary="wrap")
    
    survivors = ((world == 1) & (num_neighbors > 1) & (num_neighbors < 4))  # old cells that survive
    births    = ((world == 0) & (num_neighbors == 3))                       # new cells that are born
    return (births | survivors)

Repeatedly evaluate the next Jupyter cell (control-enter) to animate.

In [ ]:
# world = new_world()       # uncomment to reset the world

world = step_scipy(world)
show(world)